### inference.py要加上
            predicted_labels.extend(preds.cpu().numpy())
            predicted_probs.extend(probs.cpu().numpy())
            ground_truth_labels.extend(labels.cpu().numpy())

    val_acc = correct / total
    f1 = f1_score(ground_truth_labels, predicted_labels)
    try:
        auc = roc_auc_score(ground_truth_labels, predicted_probs)
    except:
        auc = float('nan')        
    
    print(f"[Valid/ Test] Acc: {val_acc:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")
    
    # Save the results to a CSV file
    results = pd.DataFrame({'study_folder': study_folders, 'predicted_prob': predicted_probs, 'predicted_label': predicted_labels, 'ground_truth_label': ground_truth_labels}, columns=['study_folder', 'predicted_prob', 'predicted_label', 'ground_truth_label'])
    csv_path = "自訂"
    # Ensure the directory exists
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    results.to_csv(csv_path, index=False)
    print(f"Classification results have been saved to: {csv_path}")

In [57]:
import pandas as pd

# 讀取結果 CSV
df = pd.read_csv(r"C:\Users\Vivo\2025_medicalimage_and_AI\test_result\ResNet34\mdResNet34_test_result.csv")
df_eff = pd.read_csv(r"C:\Users\Vivo\2025_medicalimage_and_AI\test_result\efficientnet_v2_m\mdefficientnet_v2_m_test_result.csv")

df["body_part"] = df["study_folder"].apply(lambda x: x.split("\\")[0])
df_eff["body_part"] = df_eff["study_folder"].apply(lambda x: x.split("\\")[0])

# 1. 找出錯誤預測
incorrect_preds = df[df["predicted_label"] != df["ground_truth_label"]]
incorrect_preds_eff = df_eff[df_eff["predicted_label"] != df_eff["ground_truth_label"]]

# 2. 找出正確但信心機率低預測（例如低於 0.6）
threshold = 0.6
correct_low_conf = df[
    (df["predicted_label"] == df["ground_truth_label"]) & (
        ((df["ground_truth_label"] == 1) & (df["predicted_prob"] < threshold)) |
        ((df["ground_truth_label"] == 0) & (df["predicted_prob"] > 1 - threshold))
    )
]
correct_low_conf_eff = df_eff[
    (df_eff["predicted_label"] == df_eff["ground_truth_label"]) & (
        ((df_eff["ground_truth_label"] == 1) & (df_eff["predicted_prob"] < threshold)) |
        ((df_eff["ground_truth_label"] == 0) & (df_eff["predicted_prob"] > 1 - threshold))
    )
]

# 3. 找出正確且信心機率高的預測（例如高於 0.9）
threshold_high = 0.9
correct_high_conf = df[
    (df["predicted_label"] == df["ground_truth_label"]) & (
        ((df["ground_truth_label"] == 1) & (df["predicted_prob"] > threshold_high)) |
        ((df["ground_truth_label"] == 0) & (df["predicted_prob"] < 1 - threshold_high))
    )
]
correct_high_conf_eff = df_eff[
    (df_eff["predicted_label"] == df_eff["ground_truth_label"]) & (
        ((df_eff["ground_truth_label"] == 1) & (df_eff["predicted_prob"] > threshold_high)) |
        ((df_eff["ground_truth_label"] == 0) & (df_eff["predicted_prob"] < 1 - threshold_high))
    )
]


print("錯誤預測的詳細資料:")
print("錯誤預測的數量:", len(incorrect_preds))

print("----------------------------------------")
print("正確但信心機率低的預測的詳細資料:")
print("正確但信心機率低的預測數量:", len(correct_low_conf))

print("----------------------------------------")
print("正確且信心機率高的預測的詳細資料:")  
print("正確且信心機率高的預測數量:", len(correct_high_conf))

print("----------------------------------------------------------------------------------------------")
print("eff錯誤預測的詳細資料:")
print("eff錯誤預測的數量:", len(incorrect_preds_eff))

print("----------------------------------------")
print("eff正確但信心機率低的預測的詳細資料:")
print("eff正確但信心機率低的預測數量:", len(correct_low_conf_eff))

print("----------------------------------------")
print("eff正確且信心機率高的預測的詳細資料:")  
print("eff正確且信心機率高的預測數量:", len(correct_high_conf_eff))



錯誤預測的詳細資料:
錯誤預測的數量: 219
----------------------------------------
正確但信心機率低的預測的詳細資料:
正確但信心機率低的預測數量: 58
----------------------------------------
正確且信心機率高的預測的詳細資料:
正確且信心機率高的預測數量: 475
----------------------------------------------------------------------------------------------
eff錯誤預測的詳細資料:
eff錯誤預測的數量: 205
----------------------------------------
eff正確但信心機率低的預測的詳細資料:
eff正確但信心機率低的預測數量: 53
----------------------------------------
eff正確且信心機率高的預測的詳細資料:
eff正確且信心機率高的預測數量: 531


In [58]:
def calculate_overlap(df1, df2, label):
    set1 = set(df1["study_folder"])
    set2 = set(df2["study_folder"])
    
    intersection = set1 & set2
    union = set1 | set2

    print(f"\n【{label}】")
    print(f"{label} - ResNet 數量: {len(set1)}")
    print(f"{label} - EfficientNet 數量: {len(set2)}")
    print(f"{label} - 重疊數量: {len(intersection)}")
    print(f"{label} - 重疊率（交集 / 聯集）: {len(intersection)/len(union):.2%}")
    print(f"{label} - 重疊率（交集 / ResNet）: {len(intersection)/len(set1):.2%}")
    print(f"{label} - 重疊率（交集 / EfficientNet）: {len(intersection)/len(set2):.2%}")

# 比較三個類別的圖像重疊率
calculate_overlap(incorrect_preds, incorrect_preds_eff, "錯誤預測")
calculate_overlap(correct_low_conf, correct_low_conf_eff, "正確但低信心")
calculate_overlap(correct_high_conf, correct_high_conf_eff, "正確且高信心")



【錯誤預測】
錯誤預測 - ResNet 數量: 219
錯誤預測 - EfficientNet 數量: 205
錯誤預測 - 重疊數量: 150
錯誤預測 - 重疊率（交集 / 聯集）: 54.74%
錯誤預測 - 重疊率（交集 / ResNet）: 68.49%
錯誤預測 - 重疊率（交集 / EfficientNet）: 73.17%

【正確但低信心】
正確但低信心 - ResNet 數量: 58
正確但低信心 - EfficientNet 數量: 53
正確但低信心 - 重疊數量: 5
正確但低信心 - 重疊率（交集 / 聯集）: 4.72%
正確但低信心 - 重疊率（交集 / ResNet）: 8.62%
正確但低信心 - 重疊率（交集 / EfficientNet）: 9.43%

【正確且高信心】
正確且高信心 - ResNet 數量: 475
正確且高信心 - EfficientNet 數量: 531
正確且高信心 - 重疊數量: 392
正確且高信心 - 重疊率（交集 / 聯集）: 63.84%
正確且高信心 - 重疊率（交集 / ResNet）: 82.53%
正確且高信心 - 重疊率（交集 / EfficientNet）: 73.82%


In [60]:
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
# 獲取所有出現過的部位
all_body_parts = sorted(set(df["body_part"]) | set(df_eff["body_part"]))

print("每個部位的 AUC 統計：")
print("=" * 40)
for part in all_body_parts:
    part_df_resnet = df[df["body_part"] == part]
    part_df_eff = df_eff[df_eff["body_part"] == part]
    try:
        auc_resnet = roc_auc_score(part_df_resnet["ground_truth_label"], part_df_resnet["predicted_prob"])
    except ValueError:
        auc_resnet = "無法計算（可能因只有單一類別）"
    try:
        auc_eff = roc_auc_score(part_df_eff["ground_truth_label"], part_df_eff["predicted_prob"])
    except ValueError:
        auc_eff = "無法計算（可能因只有單一類別）"
    print(f"{part}")
    print(f"  ResNet AUC       : {auc_resnet}")
    print(f"  EfficientNet AUC : {auc_eff}")
    print("-" * 40)


每個部位的 AUC 統計：
XR_ELBOW
  ResNet AUC       : 0.8774703557312253
  EfficientNet AUC : 0.9174901185770751
----------------------------------------
XR_FINGER
  ResNet AUC       : 0.8690413829229964
  EfficientNet AUC : 0.8960188580408591
----------------------------------------
XR_FOREARM
  ResNet AUC       : 0.8956068840579711
  EfficientNet AUC : 0.8838315217391304
----------------------------------------
XR_HAND
  ResNet AUC       : 0.8336333633363336
  EfficientNet AUC : 0.8123312331233123
----------------------------------------
XR_HUMERUS
  ResNet AUC       : 0.917910447761194
  EfficientNet AUC : 0.9067164179104478
----------------------------------------
XR_SHOULDER
  ResNet AUC       : 0.8441254651780968
  EfficientNet AUC : 0.8639021796916534
----------------------------------------
XR_WRIST
  ResNet AUC       : 0.9304860088365243
  EfficientNet AUC : 0.9112665684830633
----------------------------------------


In [61]:
from sklearn.metrics import accuracy_score
# 計算每個 body_part 的 AUC（處理無法計算的情況）
auc_dict_res = {}
auc_dict_eff = {}
all_parts = sorted(set(df["body_part"]) | set(df_eff["body_part"]))

for part in all_parts:
    df_r = df[df["body_part"] == part]
    df_e = df_eff[df_eff["body_part"] == part]
    try:
        auc_dict_res[part] = roc_auc_score(df_r["ground_truth_label"], df_r["predicted_prob"])
    except:
        auc_dict_res[part] = 0.5  # 或設定為 0.5，代表沒資訊
    try:
        auc_dict_eff[part] = roc_auc_score(df_e["ground_truth_label"], df_e["predicted_prob"])
    except:
        auc_dict_eff[part] = 0.5
    print(f"{part} - ResNet AUC: {auc_dict_res[part]:.4f}, EfficientNet AUC: {auc_dict_eff[part]:.4f}")

# 根據 body_part 做加權
combined = df[["study_folder", "ground_truth_label", "predicted_prob"]].copy()
combined = combined.rename(columns={"predicted_prob": "resnet_prob"})
combined["effnet_prob"] = df_eff["predicted_prob"]
combined["body_part"] = df["body_part"]

# 計算加權分數
def weighted_score(row):
    part = row["body_part"]
    auc_r = auc_dict_res.get(part, 0.5)
    auc_e = auc_dict_eff.get(part, 0.5)
    total = auc_r + auc_e
    if total == 0:
        return 0.5  # 沒有資訊
    return (auc_r * row["resnet_prob"] + auc_e * row["effnet_prob"]) / total

combined["weighted_prob"] = combined.apply(weighted_score, axis=1)

# 轉成預測 label，threshold = 0.5
combined["predicted_label"] = (combined["weighted_prob"] >= 0.5).astype(int)

# 計算 AUC 和 Accuracy
auc_final = roc_auc_score(combined["ground_truth_label"], combined["weighted_prob"])
acc_final = accuracy_score(combined["ground_truth_label"], combined["predicted_label"])

print(f"加權融合後 AUC : {auc_final:.4f}")
print(f"加權融合後 ACC : {acc_final:.4f}")


XR_ELBOW - ResNet AUC: 0.8775, EfficientNet AUC: 0.9175
XR_FINGER - ResNet AUC: 0.8690, EfficientNet AUC: 0.8960
XR_FOREARM - ResNet AUC: 0.8956, EfficientNet AUC: 0.8838
XR_HAND - ResNet AUC: 0.8336, EfficientNet AUC: 0.8123
XR_HUMERUS - ResNet AUC: 0.9179, EfficientNet AUC: 0.9067
XR_SHOULDER - ResNet AUC: 0.8441, EfficientNet AUC: 0.8639
XR_WRIST - ResNet AUC: 0.9305, EfficientNet AUC: 0.9113
加權融合後 AUC : 0.8902
加權融合後 ACC : 0.8365


In [62]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Vivo\2025_medicalimage_and_AI\test_result\ResNet34\mdResNet34_testset_result_1.csv")
df_eff = pd.read_csv(r"C:\Users\Vivo\2025_medicalimage_and_AI\test_result\efficientnet_v2_m\mdefficientnet_v2_m_testset_result_1.csv")
count = 0
for i in range(len(df_eff)):
    if df_eff["predicted_label"][i] != df["predicted_label"][i]:
        count+=1
        print(f"predicted_label不同, {df_eff['imagename'][i]}, efficientnet_v2_m: {df_eff['predicted_prob'][i]}, resnet34: {df['predicted_prob'][i]}")
print(count)

predicted_label不同, img_0002.png, efficientnet_v2_m: 0.39851642, resnet34: 0.9548655
predicted_label不同, img_0006.png, efficientnet_v2_m: 0.27849498, resnet34: 0.82179475
predicted_label不同, img_0008.png, efficientnet_v2_m: 0.11102303, resnet34: 0.6309739
predicted_label不同, img_0010.png, efficientnet_v2_m: 0.4767211, resnet34: 0.57601595
predicted_label不同, img_0015.png, efficientnet_v2_m: 0.1727916, resnet34: 0.7264963
predicted_label不同, img_0017.png, efficientnet_v2_m: 0.62551117, resnet34: 0.46621472
predicted_label不同, img_0022.png, efficientnet_v2_m: 0.3918086, resnet34: 0.819436
predicted_label不同, img_0024.png, efficientnet_v2_m: 0.93762666, resnet34: 0.12023286
predicted_label不同, img_0035.png, efficientnet_v2_m: 0.65480494, resnet34: 0.26329073
predicted_label不同, img_0055.png, efficientnet_v2_m: 0.103865504, resnet34: 0.7299158
predicted_label不同, img_0065.png, efficientnet_v2_m: 0.5780614, resnet34: 0.13487351
predicted_label不同, img_0066.png, efficientnet_v2_m: 0.74696153, resnet34: 

In [64]:
df_label = pd.read_csv(r"C:\Users\Vivo\2025_medicalimage_and_AI\MURA-v1.1\anonymized_dataset\.ipynb_checkpoints\label_mapping-checkpoint.csv")
# 建立 df_all，假設三者順序一致
df_all = pd.DataFrame({
    "imagename": df["imagename"],
    "resnet_pred": df["predicted_label"],
    "resnet_prob": df["predicted_prob"],
    "effnet_pred": df_eff["predicted_label"],
    "effnet_prob": df_eff["predicted_prob"],
    "label": df_label["label"],
    "body_part": df_label["body_part"]
})
print(df_all.head(10))

      imagename  resnet_pred  resnet_prob  effnet_pred  effnet_prob  label  \
0  img_0001.png            1     0.980427            1     0.945519      1   
1  img_0002.png            1     0.954866            0     0.398516      1   
2  img_0003.png            0     0.001798            0     0.033258      0   
3  img_0004.png            0     0.409477            0     0.130181      1   
4  img_0005.png            1     0.995130            1     0.995233      1   
5  img_0006.png            1     0.821795            0     0.278495      1   
6  img_0007.png            0     0.070657            0     0.357550      0   
7  img_0008.png            1     0.630974            0     0.111023      0   
8  img_0009.png            0     0.155674            0     0.094460      0   
9  img_0010.png            1     0.576016            0     0.476721      1   

     body_part  
0  XR_SHOULDER  
1  XR_SHOULDER  
2    XR_FINGER  
3      XR_HAND  
4  XR_SHOULDER  
5   XR_FOREARM  
6     XR_ELBOW  
7    

In [65]:
from sklearn.metrics import accuracy_score, roc_auc_score
# Ground truth
y_true = df_all["label"]

# ResNet 預測
resnet_pred = df_all["resnet_pred"]
resnet_prob = df_all["resnet_prob"]

# EfficientNet 預測
effnet_pred = df_all["effnet_pred"]
effnet_prob = df_all["effnet_prob"]

# 計算 accuracy
resnet_acc = accuracy_score(y_true, resnet_pred)
effnet_acc = accuracy_score(y_true, effnet_pred)

# 計算 AUC（需要機率預測）
resnet_auc = roc_auc_score(y_true, resnet_prob)
effnet_auc = roc_auc_score(y_true, effnet_prob)

# 顯示結果
print(f"ResNet - Accuracy: {resnet_acc:.4f}, AUC: {resnet_auc:.4f}")
print(f"EfficientNet - Accuracy: {effnet_acc:.4f}, AUC: {effnet_auc:.4f}")

ResNet - Accuracy: 0.7961, AUC: 0.8761
EfficientNet - Accuracy: 0.8211, AUC: 0.8710


In [67]:

# 加入 weighted score 計算到 df_all 中
def weighted_score(row):
    part = row["body_part"]
    auc_r = auc_dict_res.get(part, 0.5)
    auc_e = auc_dict_eff.get(part, 0.5)
    total = auc_r + auc_e
    if total == 0:
        return 0.5  # 沒有資訊
    return (auc_r * row["resnet_prob"] + auc_e * row["effnet_prob"]) / total

# 新增 weighted_prob 欄位
df_all["weighted_prob"] = df_all.apply(weighted_score, axis=1)

# 計算預測結果（使用閾值 0.5）
df_all["predicted_label"] = (df_all["weighted_prob"] >= 0.5).astype(int)

# 整體融合後的 AUC 和 ACC
auc_final = roc_auc_score(df_all["label"], df_all["weighted_prob"])
acc_final = accuracy_score(df_all["label"], df_all["predicted_label"])

print(f"加權融合後 AUC : {auc_final:.4f}")
print(f"加權融合後 ACC : {acc_final:.4f}")


加權融合後 AUC : 0.8883
加權融合後 ACC : 0.8322


In [68]:
print(df_all.head(10))

      imagename  resnet_pred  resnet_prob  effnet_pred  effnet_prob  label  \
0  img_0001.png            1     0.980427            1     0.945519      1   
1  img_0002.png            1     0.954866            0     0.398516      1   
2  img_0003.png            0     0.001798            0     0.033258      0   
3  img_0004.png            0     0.409477            0     0.130181      1   
4  img_0005.png            1     0.995130            1     0.995233      1   
5  img_0006.png            1     0.821795            0     0.278495      1   
6  img_0007.png            0     0.070657            0     0.357550      0   
7  img_0008.png            1     0.630974            0     0.111023      0   
8  img_0009.png            0     0.155674            0     0.094460      0   
9  img_0010.png            1     0.576016            0     0.476721      1   

     body_part  weighted_prob  predicted_label  
0  XR_SHOULDER       0.962770                1  
1  XR_SHOULDER       0.673470              

In [71]:
# 選取需要的欄位並重新命名（如果你想要 column 名稱更直觀）
df_output = df_all[["imagename", "predicted_label", "weighted_prob"]].copy()
df_output.columns = ["filename", "predicted_class", "predicted_prob"]

# 輸出為 CSV
df_output.to_csv(r"C:\Users\Vivo\2025_medicalimage_and_AI\final預測結果.csv", index=False)
print("預測結果已輸出到 final_predictions.csv")


預測結果已輸出到 final_predictions.csv
